In [ ]:
# IMPORT ALL YOUR LIBRARIES
!pip install transformers
!pip install datasets
!pip install accelerate >=0.21.0
!pip install transformers[torch]

import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from datasets import Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
import accelerate

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import re

In [ ]:
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/sun.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/deu.csv"
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/deu_c.csv"
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/dev/sun_c.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/dev/rus_c.csv"

File ‘/kaggle/working/deu.csv’ already there; not retrieving.

File ‘/kaggle/working/rus_c.csv’ already there; not retrieving.



In [ ]:
df = pd.read_csv('/kaggle/working/deu.csv')
df.head()

id                                               text  \
0  deu_train_track_a_00001  Nein nein nix da, fuck den schön weiter ab bis...   
1  deu_train_track_a_00002  Vor 100 Jahren ging man auf die Strasse weil m...   
2  deu_train_track_a_00003                Wann endlich Gabby Epstein heiraten   
3  deu_train_track_a_00004  Wegen dem Song 1001 Nacht, dem einen Phantomto...   
4  deu_train_track_a_00005  Dann sollten wir versuchen, sie zu erreichen. ...   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0      1        0     0    0        0         0  
1      1        1     0    0        0         0  
2      0        0     0    0        0         0  
3      0        0     0    1        0         0  
4      0        0     0    0        0         0

In [ ]:
df_test = pd.read_csv('/kaggle/working/rus_c.csv')
df_test.head()

id                                               text  \
0  rus_dev_track_c_00001                           как всегда хуйни зададут   
1  rus_dev_track_c_00002  Никто не знает силы своих способностей, пока о...   
2  rus_dev_track_c_00003                           Ого!!) 62 с первого раза   
3  rus_dev_track_c_00004  Блин! Могу сидеть только в твиттере, а так хоч...   
4  rus_dev_track_c_00005           Даа! Я это сделала!! Мама уезжает!))))))   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0    NaN      NaN   NaN  NaN      NaN       NaN  
1    NaN      NaN   NaN  NaN      NaN       NaN  
2    NaN      NaN   NaN  NaN      NaN       NaN  
3    NaN      NaN   NaN  NaN      NaN       NaN  
4    NaN      NaN   NaN  NaN      NaN       NaN

In [ ]:
# from bs4 import BeautifulSoup
# import re

# # # Step 2: Clean the text data
# # def clean_text(text):
# #     # Remove HTML tags
# #     text = BeautifulSoup(text, "html.parser").get_text()
# #     # Remove numbers (but keep special characters)
# #     text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
# #     # Convert to lowercase
# #     text = text.lower()
# #     # Remove extra whitespace
# #     text = ' '.join(text.split())
# #     return text

# # # Apply the clean_text function
# # df_test['text'] = df_test['text'].apply(clean_text)

# # Step 3-5: (Same as your original code)
# df_test = df_test[df_test['text'].notna() & (df_test['text'] != '')]

# # Ensure emotion columns are integers
# # emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotion_columns = ['Anger', 'Disgust','Fear', 'Joy', 'Sadness', 'Surprise']
# df_test[emotion_columns] = df_test[emotion_columns]

# # Save the processed data back to CSV
# output_path = '/kaggle/working/cleaned_test_english.csv'
# df_test.to_csv(output_path, index=False)

# print(f"Data preprocessing completed. File 'cleaned_test_english.csv' has been created in the /kaggle/working directory.")

# Display statistics and processed examples
print(f"Total samples: {len(df_test)}")
print("\nEmotion distribution in the dataset:")
print(df_test[emotion_columns].sum())
print("\nFirst few processed examples:")
print(df_test.head())


Total samples: 401

Emotion distribution in the dataset:
Anger       0.0
Disgust     0.0
Fear        0.0
Joy         0.0
Sadness     0.0
Surprise    0.0
dtype: float64

First few processed examples:
                      id                                               text  \
0  rus_dev_track_c_00001                           как всегда хуйни зададут   
1  rus_dev_track_c_00002  Никто не знает силы своих способностей, пока о...   
2  rus_dev_track_c_00003                           Ого!!) 62 с первого раза   
3  rus_dev_track_c_00004  Блин! Могу сидеть только в твиттере, а так хоч...   
4  rus_dev_track_c_00005           Даа! Я это сделала!! Мама уезжает!))))))   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0    NaN      NaN   NaN  NaN      NaN       NaN  
1    NaN      NaN   NaN  NaN      NaN       NaN  
2    NaN      NaN   NaN  NaN      NaN       NaN  
3    NaN      NaN   NaN  NaN      NaN       NaN  
4    NaN      NaN   NaN  NaN      NaN       NaN  


In [ ]:
if 'id' in df.columns:
    df = df.drop(['id'], axis=1)
from bs4 import BeautifulSoup
import re

# # Step 2: Clean the text data
# def clean_text(text):
#     # Remove HTML tags
#     text = BeautifulSoup(text, "html.parser").get_text()
#     # Remove numbers (but keep special characters)
#     text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
#     # Convert to lowercase
#     text = text.lower()
#     # Remove extra whitespace
#     text = ' '.join(text.split())
#     return text

# # df['text'] = df['text'].apply(clean_text)

# Step 3: Remove rows with empty text
df = df[df['text'].notna() & (df['text'] != '')]

# Step 4: Ensure emotion columns are integers
emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# emotion_columns = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
df[emotion_columns] = df[emotion_columns].astype(int)

# Step 5: Save the processed data back to CSV
output_path = '/kaggle/working/cleaned_deu.csv'
df.to_csv(output_path, index=False)

print(f"Data preprocessing completed. File 'cleaned_som.csv' has been created in the /kaggle/working directory.")

# Optional: Display some statistics about the data
print(f"Total samples: {len(df)}")
print("\nEmotion distribution in the dataset:")
print(df[emotion_columns].sum())

# Optional: Display a few processed examples
print("\nFirst few processed examples:")
print(df.head())

Data preprocessing completed. File 'cleaned_som.csv' has been created in the /kaggle/working directory.
Total samples: 2603

Emotion distribution in the dataset:
Anger       768
Disgust     832
Fear        239
Joy         541
Sadness     516
Surprise    159
dtype: int64

First few processed examples:
                                                text  Anger  Disgust  Fear  \
0  Nein nein nix da, fuck den schön weiter ab bis...      1        0     0   
1  Vor 100 Jahren ging man auf die Strasse weil m...      1        1     0   
2                Wann endlich Gabby Epstein heiraten      0        0     0   
3  Wegen dem Song 1001 Nacht, dem einen Phantomto...      0        0     0   
4  Dann sollten wir versuchen, sie zu erreichen. ...      0        0     0   

   Joy  Sadness  Surprise  
0    0        0         0  
1    0        0         0  
2    0        0         0  
3    1        0         0  
4    0        0         0  


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.metrics import f1_score

# Use DistilBERT model
MODEL_NAME = "distilbert-base-multilingual-cased"

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

# Load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=6)  # 6 emotions

# Example to confirm the model and tokenizer are loaded
print("Model and tokenizer loaded successfully.")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and tokenizer loaded successfully.


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
train_dataset = Dataset.from_pandas(df)

train_dataset

dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
val_dataset = dataset['test']
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise'],
        num_rows: 2082
    })
    test: Dataset({
        features: ['text', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise'],
        num_rows: 521
    })
})


In [ ]:
# Convert train_dataset and val_dataset back to Pandas DataFrame
train_df = train_dataset.to_pandas()
val_df = val_dataset.to_pandas()

# Save to CSV
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)

print("CSV files have been saved.")


CSV files have been saved.


In [ ]:
train_df.head()

text  Anger  Disgust  Fear  \
0  Es ist einfach unfassbar, dass man sich in Deu...      1        0     1   
1  Ich habe mir bei der Geburt meiner Tochter ein...      0        0     0   
2  Als ich meinem Arbeitgeber von der Schwangersc...      0        0     0   
3  Habe meist alle Türen zu, sofern ich nicht ger...      0        0     1   
4  Das gibt es in **delikat** und **pikant**, abe...      0        0     0   

   Joy  Sadness  Surprise  
0    0        1         0  
1    1        0         0  
2    0        0         0  
3    0        0         0  
4    0        0         0

In [ ]:
val_df.head()

text  Anger  Disgust  Fear  \
0  Akupunturen und K-Tape koennen vielleicht helf...      0        0     0   
1  Ich habe mir ein einziges Stilltop gekauft und...      0        1     0   
2  Wenn die Vollidioten aus der Politik verschwun...      1        1     0   
3  Man kann leider Friedrich Merz nicht im Hinter...      1        1     0   
4  ich hatte mal was mit einer und ich hab ihr da...      0        0     0   

   Joy  Sadness  Surprise  
0    0        0         0  
1    0        0         0  
2    0        0         0  
3    0        0         0  
4    1        0         0

In [ ]:
# #print the values in train_dataset

# print(train_dataset[0])

In [ ]:
# #for each entry in train_dataset, merge Anger, Fear, Joy, Sadness, Surprise

# train_dataset = train_dataset.map(lambda x: {'labels': str(x['Anger']) + ' + '  + str(x['Fear']) + ' + '  +  str(x['Joy']) + ' + '  + str(x['Sadness']) +  ' + ' + str(x['Surprise'])})

# #drop Anger Fear Joy Sadness and Surprise

# train_dataset = train_dataset.remove_columns(['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])

# #print the labels coloumn to check

# print(train_dataset)

# #print labels
# print(train_dataset[0])

In [ ]:
# #do the same for validation dataset

# val_dataset = val_dataset.map(lambda x: {'labels': str(x['Anger']) + ' + '  + str(x['Fear']) + ' + '  +  str(x['Joy']) + ' + '  + str(x['Sadness']) +  ' + ' + str(x['Surprise'])})

# val_dataset = val_dataset.remove_columns(['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])

# print(val_dataset)

# #print labels
# print(val_dataset[0])

In [ ]:
cols = train_df.columns[1:]
cols

Index(['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise'], dtype='object')

In [ ]:
training_labels = train_df[cols]
training_labels = training_labels.to_numpy()
training_labels = training_labels.astype(np.float32)

train_sentences = train_df.text.values.tolist()

validation_labels = val_df[cols]
validation_labels = validation_labels.to_numpy()
validation_labels = validation_labels.astype(np.float32)

validation_sentences = val_df.text.values.tolist()

In [ ]:
# training_sentence_encoding = tokenizer(train_sentences, padding = True, truncation = True, max_length = 512)
# validation_sentence_encoding = tokenizer(validation_sentences, padding = True, truncation = True, max_length = 512)

train_encodings = tokenizer(train_sentences, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(validation_sentences, padding=True, truncation=True, max_length=512)

In [ ]:
# #defining a custom dataset class for the training and validation datasets to be used in the trainer

# class ArgumentDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# #creating the training and validation datasets
# train_dataset_new = ArgumentDataset(training_sentence_encoding, training_labels)
# validation_dataset = ArgumentDataset(validation_sentence_encoding, validation_labels)

# #train_dataset[0]

In [ ]:
from datasets import Dataset
import torch
import numpy as np

train_dataset_new = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': training_labels
})

validation_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': validation_labels
})

In [ ]:
# print(type(train_dataset_new))

In [ ]:
# print(train_dataset_new[0])

In [ ]:
# source = train_dataset['text']
# target = train_dataset['labels']

# max_length = tokenizer.model_max_length

# tokenizer.model_max_length = max_length

In [ ]:
# def tokenize_function(examples):
#     tokenized_inputs = tokenizer(
#         examples["text"],
#         padding="max_length",  # Pad to the max length
#         truncation=True,       # Truncate to avoid exceeding model limits
#         max_length=max_length
#     )

#     tokenized_labels = tokenizer(
#         examples["labels"],
#         padding="max_length",  # Pad to the max length
#         truncation=True,       # Truncate labels as well
#         max_length=max_length
#     )

#     # Ensure that tokenized_labels are added as 'labels'
#     tokenized_inputs["labels"] = examples["labels"]

#     return tokenized_inputs



# # Tokenize the training and validation datasets
# tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
# val_dataset = val_dataset.map(tokenize_function, batched=True)

# # Format the dataset for training
# tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# print(tokenized_dataset[0])

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

epochs = 4
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(df) * epochs)

training_arguments = TrainingArguments(
    output_dir="./outputs",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_strategy="epoch"
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import wandb

# Initialize wandb with mode disabled
wandb.init(mode="disabled")

In [ ]:
#defining the metrics to be calculated during training
def multi_label_metrics(predictions, labels,):
    sigmoid = torch.nn.Sigmoid()
    predictions = sigmoid(torch.tensor(predictions))
    y_predictions = np.zeros(predictions.shape)
    y_true = labels
    y_predictions[np.where(predictions >= 0.4)] = 1
    accuracy = accuracy_score(y_true, y_predictions)
    f1 = f1_score(y_true, y_predictions, average='micro')
    auc = roc_auc_score(y_true, y_predictions, average='micro')

    metrics = {
        'accuracy': accuracy,
        'f1': f1,
        'auc': auc
    }

    return metrics

#defining the function to compute the metrics for each batch of data
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(preds, p.label_ids)

    return result

#defining the trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset_new,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

TrainOutput(global_step=524, training_loss=0.3683179234730378, metrics={'train_runtime': 244.219, 'train_samples_per_second': 34.101, 'train_steps_per_second': 2.146, 'total_flos': 471905302490208.0, 'train_loss': 0.3683179234730378, 'epoch': 4.0})

In [ ]:
#Saving the model
model.save_pretrained("/content/")

In [ ]:
#checking the training history of the model
trainer.state.log_history

[{'eval_loss': 0.4197482764720917,
  'eval_accuracy': 0.29942418426103645,
  'eval_f1': 0.4022222222222222,
  'eval_auc': 0.6269192990727646,
  'eval_runtime': 3.077,
  'eval_samples_per_second': 169.323,
  'eval_steps_per_second': 10.725,
  'epoch': 1.0,
  'step': 131},
 {'eval_loss': 0.4055948257446289,
  'eval_accuracy': 0.36084452975047987,
  'eval_f1': 0.4909456740442656,
  'eval_auc': 0.6727487034417727,
  'eval_runtime': 3.1245,
  'eval_samples_per_second': 166.745,
  'eval_steps_per_second': 10.562,
  'epoch': 2.0,
  'step': 262},
 {'eval_loss': 0.40268367528915405,
  'eval_accuracy': 0.3723608445297505,
  'eval_f1': 0.5189873417721519,
  'eval_auc': 0.694536775106082,
  'eval_runtime': 3.1288,
  'eval_samples_per_second': 166.516,
  'eval_steps_per_second': 10.547,
  'epoch': 3.0,
  'step': 393},
 {'loss': 0.3718,
  'grad_norm': 1.566973090171814,
  'learning_rate': 2.2900763358778625e-06,
  'epoch': 3.816793893129771,
  'step': 500},
 {'eval_loss': 0.39283397793769836,
  'eva

In [ ]:
import torch

# Load the model and tokenizer
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load the DistilBERT model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained("/content/", num_labels=6)  # Path to the model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')  # Tokenizer for DistilBERT

# model = AutoModelForSequenceClassification.from_pretrained("/content/", num_labels=6)
# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")


# MODEL_NAME = "xlm-roberta-base"  # Change to "bert-base-multilingual-cased" for mBERT
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=6)  # 6 emotions

# Make sure to set the model to evaluation mode
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
# Test data (assuming you have a DataFrame with a column 'text')
test_sentences = df_test['text'].tolist()

# Tokenize the test data
test_encodings = tokenizer(test_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
# Move the encodings to the same device as the model (GPU if available, otherwise CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Pass the test data through the model to get predictions
with torch.no_grad():
    input_ids = test_encodings['input_ids'].to(device)
    attention_mask = test_encodings['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)

# Apply sigmoid to get probabilities
sigmoid = torch.nn.Sigmoid()
predictions = sigmoid(outputs.logits)

# Convert probabilities to binary predictions (assuming a threshold of 0.5)
predicted_labels = (predictions > 0.5).int()

# Convert predictions to a more readable format (back to CPU if necessary)
predicted_labels = predicted_labels.cpu().numpy()


In [ ]:
# Assuming your labels are in the order ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
label_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']

# Convert predictions to a DataFrame for easy viewing
predicted_df = pd.DataFrame(predicted_labels, columns=label_columns)

# Combine with the original test sentences
result_df = pd.concat([df_test['text'], predicted_df], axis=1)

# Display the result
print(result_df.head())


                                                text  Anger  Disgust  Fear  \
0                           как всегда хуйни зададут      0        0     0   
1  Никто не знает силы своих способностей, пока о...      0        0     0   
2                           Ого!!) 62 с первого раза      0        0     0   
3  Блин! Могу сидеть только в твиттере, а так хоч...      0        0     0   
4           Даа! Я это сделала!! Мама уезжает!))))))      0        0     0   

   Joy  Sadness  Surprise  
0    0        0         0  
1    0        0         0  
2    0        0         0  
3    1        0         0  
4    1        0         0  


In [ ]:
# Save predictions to CSV
result_df.to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'.")


Predictions saved to 'predictions.csv'.


In [ ]:
import pandas as pd

# Step 1: Load the predictions CSV file
df = pd.read_csv('predictions.csv')

# Step 2: Replace 'text' column with an 'id' column
df['id'] = [f"rus_dev_track_c_{str(i+1).zfill(5)}" for i in range(len(df))]

# Step 3: Drop the original 'text' column
df = df.drop(columns=['text'])

# Step 4: Reorder columns so 'id' comes first, followed by the emotions in the correct order
df = df[['id',  'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# Step 5: Save the updated CSV file for submission
df.to_csv('pred_rus_c.csv', index=False)

print("Submission file 'final_submission.csv' has been created.")


Submission file 'final_submission.csv' has been created.


In [ ]:
# import numpy as np
# import torch
# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# # Defining the metrics to be calculated during training
# def multi_class_multi_label_metrics(predictions, labels):
#     sigmoid = torch.nn.Sigmoid()
#     predictions = sigmoid(torch.tensor(predictions))

#     # Ensure predictions are rounded to 0 and 1
#     predictions_rounded = np.round(predictions.detach().numpy())  # Convert to numpy and round

#     # Use labels directly if they are already numpy arrays
#     y_true = labels  # Assuming labels are in numpy format

#     # Check shapes for debugging
#     print("Rounded Predictions shape:", predictions_rounded.shape)
#     print("True Labels shape:", y_true.shape)

#     # Calculate metrics
#     accuracy = accuracy_score(y_true, predictions_rounded)
#     f1 = f1_score(y_true, predictions_rounded, average='micro')
#     auc = roc_auc_score(y_true, predictions_rounded, average='micro')

#     metrics = {
#         'accuracy': accuracy,
#         'f1': f1,
#         'auc': auc
#     }

#     return metrics

# # Defining the function to compute the metrics for each batch of data
# def compute_metrics(p):
#     preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
#     result = multi_class_multi_label_metrics(preds, p.label_ids)

#     return result
